In [47]:
import torch
from cci.utils import project_dir

device = torch.device("cpu")

model = torch.jit.load(
    project_dir() / "results/CNN/137/0_model.pt", map_location=device
)
model.eval()
model

RecursiveScriptModule(
  original_name=CNNModel
  (sequential): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv1d)
    (1): RecursiveScriptModule(original_name=BatchNorm1d)
    (2): RecursiveScriptModule(original_name=Dropout)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=Conv1d)
    (5): RecursiveScriptModule(original_name=BatchNorm1d)
    (6): RecursiveScriptModule(original_name=Dropout)
    (7): RecursiveScriptModule(original_name=ReLU)
    (8): RecursiveScriptModule(original_name=AdaptiveAvgPool1d)
    (9): RecursiveScriptModule(original_name=Flatten)
    (10): RecursiveScriptModule(original_name=Linear)
  )
)

In [48]:
import os
from pathlib import Path
import numpy as np
import scipy

signal_path = Path(os.environ["OOCHA_DIR"]) / "a_150.mat"

signal = scipy.io.loadmat(signal_path, simplify_cells=True)["SIGNALS"][
    "ecg_diff"
].astype(np.float32)

In [51]:
import rerun as rr

window_size = 1500
overlap = 1


rr.init("inference", spawn=True)

predictions = []
for i in range(0, len(signal)):  # - window_size + 1, overlap):
    rr.set_time_sequence("step", i)
    rr.log("/Signal/0", rr.Scalar(signal[i]))
    if i > window_size:
        window = torch.tensor(signal[i - window_size : i]).unsqueeze(0)
        prediction = torch.nn.functional.sigmoid(model(window))
        # predictions.append(prediction.detach())

        rr.log("/pred/0", rr.Scalar(prediction.detach().item()))
    else:
        rr.log("/pred/0", rr.Scalar(0.5))

[2024-04-10T10:55:39Z INFO  egui_wgpu] There were 2 available wgpu adapters: {backend: Vulkan, device_type: IntegratedGpu, name: "AMD Radeon Graphics (RADV RENOIR)", driver: "radv", driver_info: "Mesa 23.1.9", vendor: 0x1002, device: 0x1638}, {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 16.0.6, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 23.1.9 (LLVM 16.0.6)", vendor: 0x10005}
[2024-04-10T10:55:39Z INFO  tracing::span] perform;
[2024-04-10T10:55:39Z INFO  zbus::handshake] write_command; command=Auth(Some(External), Some([49, 48, 48, 48]))
[2024-04-10T10:55:39Z INFO  tracing::span] read_command;
[2024-04-10T10:55:39Z INFO  zbus::handshake] write_command; command=NegotiateUnixFD
[2024-04-10T10:55:39Z INFO  tracing::span] read_command;
[2024-04-10T10:55:39Z INFO  zbus::handshake] write_command; command=Begin
[2024-04-10T10:55:39Z INFO  tracing::span] socket reader;
[2024-04-10T10:55:39Z INFO  tracing::span] perform;
[2024-04-10T10:55:39Z INFO  zbus::handshake] write_co

KeyboardInterrupt: 